In [1]:
library(GO.db)
library(AnnotationDbi)

Loading required package: AnnotationDbi

Loading required package: stats4

Loading required package: BiocGenerics


Attaching package: ‘BiocGenerics’


The following objects are masked from ‘package:stats’:

    IQR, mad, sd, var, xtabs


The following objects are masked from ‘package:base’:

    anyDuplicated, aperm, append, as.data.frame, basename, cbind,
    colnames, dirname, do.call, duplicated, eval, evalq, Filter, Find,
    get, grep, grepl, intersect, is.unsorted, lapply, Map, mapply,
    match, mget, order, paste, pmax, pmax.int, pmin, pmin.int,
    Position, rank, rbind, Reduce, rownames, sapply, setdiff, sort,
    table, tapply, union, unique, unsplit, which.max, which.min


Loading required package: Biobase

Welcome to Bioconductor

    Vignettes contain introductory material; view with
    'browseVignettes()'. To cite Bioconductor, see
    'citation("Biobase")', and for packages 'citation("pkgname")'.


Loading required package: IRanges

Loading required package: S4Vectors

In [77]:
GO.db::GOMFPARENTS

MFPARENTS map for GO (object of class "AnnDbBimap")

In [87]:
# Function to calculate the depth of a GO term
get_go_depth <- function(go_id,parent_type) {
  term <- go_id
  depth <- 0
  
  while (!is.na(term) && length(term) > 0) {
    parents <- AnnotationDbi::mget(term, parent_type, ifnotfound = NA)
    
    # Check if parents is NULL or empty
    if (is.null(parents) || length(parents) == 0 || all(is.na(parents))) {
      break
    }
    
    parents <- unlist(parents)
    term <- parents[1]  # Assuming single inheritance for simplicity
    depth <- depth + 1
  }
  
  return(depth)
}



# Example usage
go_id <- "GO:0008150"  # Replace with the GO term of interest
depth <- get_go_depth(go_id, parent_type = GO.db::GOBPPARENTS)
cat("The depth of GO term", go_id, "is", depth, "\n")

The depth of GO term GO:0008150 is 1 


In [79]:
go_depth<- function(go_terms,parent_type = GO.db::GOBPPARENTS) {
    go_ids <- regmatches(go_terms, regexpr("GO:\\d+", go_terms))
    depths <- numeric(length(go_ids))
    for (i in seq_along(go_ids)) {
      depths[i] <- get_go_depth(go_ids[i],parent_type)
    }
    df_depth <- data.frame(term = go_terms, go_id = go_ids, go_depth = depths)
    return (df_depth)
} 

In [65]:
go_bp=readr::read_tsv('libs_human/gmt/GO_Biological_Process_2023.txt', col_names= FALSE)
df_depth<-go_depth(go_bp$X1,parent_type = GO.db::GOBPPARENTS)

In [61]:
head(df_depth)

,term,go_id,go_depth
,<chr>,<chr>,<dbl>
1,'De Novo' AMP Biosynthetic Process (GO:0044208),GO:0044208,12
2,'De Novo' Post-Translational Protein Folding (GO:0051084),GO:0051084,5
3,2-Oxoglutarate Metabolic Process (GO:0006103),GO:0006103,8
4,3'-UTR-mediated mRNA Destabilization (GO:0061158),GO:0061158,9
5,3'-UTR-mediated mRNA Stabilization (GO:0070935),GO:0070935,10
6,3'-Phosphoadenosine 5'-Phosphosulfate Metabolic Process (GO:0050427),GO:0050427,5


In [62]:
write.table(df_depth, file = "libs_human/go_depth/GO_Biological_Process_2023.txt", sep = "\t", quote = FALSE, row.names = FALSE)

In [80]:
go_cc=readr::read_tsv('libs_human/gmt/GO_Cellular_Component_2023.txt', col_names= FALSE)
df_depth<-go_depth(go_cc$X1,parent_type = GO.db::GOCCPARENTS)

Warning message:
“One or more parsing issues, call `problems()` on your data frame for details,
e.g.:
  dat <- vroom(...)
  problems(dat)”
Rows: 474 Columns: 10
── Column specification ────────────────────────────────────────────────────────
Delimiter: "\t"
chr (9): X1, X3, X4, X5, X6, X7, X8, X9, X10
lgl (1): X2

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


In [81]:
head(df_depth)

,term,go_id,go_depth
,<chr>,<chr>,<dbl>
1,9+0 Non-Motile Cilium (GO:0097731),GO:0097731,7
2,9+2 Motile Cilium (GO:0097729),GO:0097729,7
3,90S Preribosome (GO:0030686),GO:0030686,5
4,AMPA Glutamate Receptor Complex (GO:0032281),GO:0032281,8
5,AP-1 Adaptor Complex (GO:0030121),GO:0030121,7
6,AP-2 Adaptor Complex (GO:0030122),GO:0030122,9


In [82]:
write.table(df_depth, file = "libs_human/go_depth/GO_Cellular_Component_2023.txt", sep = "\t", quote = FALSE, row.names = FALSE)

In [83]:
go_mf=readr::read_tsv('libs_human/gmt/GO_Molecular_Function_2023.txt', col_names= FALSE)
df_depth<-go_depth(go_mf$X1,parent_type = GO.db::GOMFPARENTS)

Warning message:
“One or more parsing issues, call `problems()` on your data frame for details,
e.g.:
  dat <- vroom(...)
  problems(dat)”
Rows: 1147 Columns: 21
── Column specification ────────────────────────────────────────────────────────
Delimiter: "\t"
chr (20): X1, X3, X4, X5, X6, X7, X8, X9, X10, X11, X12, X13, X14, X15, X16,...
lgl  (1): X2

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


In [84]:
head(df_depth)

,term,go_id,go_depth
,<chr>,<chr>,<dbl>
1,1-Acylglycerol-3-Phosphate O-acyltransferase Activity (GO:0003841),GO:0003841,8
2,1-Acylglycerophosphocholine O-acyltransferase Activity (GO:0047184),GO:0047184,7
3,1-Alkyl-2-Acetylglycerophosphocholine Esterase Activity (GO:0003847),GO:0003847,6
4,1-Phosphatidylinositol Binding (GO:0005545),GO:0005545,6
5,1-Phosphatidylinositol-3-Kinase Activity (GO:0016303),GO:0016303,8
6,1-Phosphatidylinositol-3-Kinase Regulator Activity (GO:0046935),GO:0046935,5


In [85]:
write.table(df_depth, file = "libs_human/go_depth/GO_Molecular_Function_2023.txt", sep = "\t", quote = FALSE, row.names = FALSE)